In [349]:
import pickle

# Open and load the routerbench_5shot.pkl file
with open('routerbench_5shot.pkl', 'rb') as f:
    routerbench_data = pickle.load(f)


/var/folders/42/5cczn_jx4454n6fvhcg5qbjw0000gn/T/ipykernel_44549/1371555631.py:5: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  routerbench_data = pickle.load(f)


In [350]:
import pandas as pd

# Configure pandas to show all columns
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)


In [351]:
routerbench_data['eval_name'].unique()

array(['Chinese_character_riddles', 'abstract2title', 'accounting_audit',
       'arc-challenge', 'bias_detection', 'chinese-lantern-riddles',
       'chinese-remainder-theorem',
       'chinese_ancient_masterpieces_dynasty', 'chinese_ancient_poetry',
       'chinese_chu_ci', 'chinese_famous_novel',
       'chinese_hard_translations', 'chinese_homonym', 'chinese_idioms',
       'chinese_modern_poem_identification', 'chinese_poem',
       'chinese_shi_jing', 'chinese_song_ci', 'chinese_tang_poetries',
       'chinese_zodiac', 'consensus_summary', 'grade-school-math',
       'hellaswag', 'mbpp', 'mmlu-abstract-algebra', 'mmlu-anatomy',
       'mmlu-astronomy', 'mmlu-business-ethics',
       'mmlu-clinical-knowledge', 'mmlu-college-biology',
       'mmlu-college-chemistry', 'mmlu-college-computer-science',
       'mmlu-college-mathematics', 'mmlu-college-medicine',
       'mmlu-college-physics', 'mmlu-computer-security',
       'mmlu-conceptual-physics', 'mmlu-econometrics',
       'mmlu-

In [352]:
routerbench_data.query('eval_name.str.startswith("mbpp")')

,sample_id,prompt,eval_name,WizardLM/WizardLM-13B-V1.2,claude-instant-v1,claude-v1,claude-v2,gpt-3.5-turbo-1106,gpt-4-1106-preview,meta/code-llama-instruct-34b-chat,meta/llama-2-70b-chat,mistralai/mistral-7b-chat,mistralai/mixtral-8x7b-chat,zero-one-ai/Yi-34B-Chat,gpt-3.5-turbo-1106|model_response,claude-instant-v1|model_response,claude-v1|model_response,claude-v2|model_response,gpt-4-1106-preview|model_response,meta/llama-2-70b-chat|model_response,mistralai/mixtral-8x7b-chat|model_response,zero-one-ai/Yi-34B-Chat|model_response,WizardLM/WizardLM-13B-V1.2|model_response,meta/code-llama-instruct-34b-chat|model_response,mistralai/mistral-7b-chat|model_response,gpt-3.5-turbo-1106|total_cost,claude-instant-v1|total_cost,claude-v1|total_cost,claude-v2|total_cost,gpt-4-1106-preview|total_cost,meta/llama-2-70b-chat|total_cost,mistralai/mixtral-8x7b-chat|total_cost,zero-one-ai/Yi-34B-Chat|total_cost,WizardLM/WizardLM-13B-V1.2|total_cost,meta/code-llama-instruct-34b-chat|total_cost,mistralai/mistral-7b-chat|total_cost,oracle_model_to_route_to
20692,mbpp.dev.0,['Write a function to find the shared elements...,mbpp,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,"[""Here's a Python function to find the shared ...",['Here is a function to find the shared elemen...,['Here is a function to find the shared elemen...,['Here is a Python function to find the shared...,['Certainly! Below is a Python function that t...,[' Sure! Here\'s a function that finds the sha...,"[""Sure, here is a simple function in Python th...","[""\n```python\ndef find_shared_elements(list1,...","[""Here's a Python function that finds the shar...","[' \n[PYTHON]\ndef find_shared_elements(list1,...",['Here is a function written in Python that ca...,0.000386,0.000486,0.003928,0.005416,0.00830,0.000262,0.000123,0.000152,0.000073,0.000172,0.000050,mistralai/mistral-7b-chat
20693,mbpp.dev.1,['Write a python function to identify non-prim...,mbpp,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,['Here\'s a Python function to identify non-pr...,['Here is a Python function to identify non-pr...,['Here is a Python function to identify non-pr...,['Here is a python function to identify non-pr...,['Here\'s a Python function that identifies no...,"["" Sure! Here's a simple function that takes a...","['Sure, here is a simple Python function that ...","[""\nHere's a simple Python function that takes...","[""Here's a simple Python function that identif...",[' \n[PYTHON]\ndef is_prime(n):\n if n <= 1...,['Here is a simple Python function that identi...,0.000423,0.000359,0.005368,0.006160,0.01091,0.000302,0.000033,0.000252,0.000082,0.000403,0.000046,mistralai/mistral-7b-chat
20694,mbpp.dev.10,['Write a function to that returns true if the...,mbpp,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,['Here\'s a function in Python that checks for...,"['def check_string(input_string):\n if ""_"" ...","[""Here is a function that returns true if the ...",['Here is a Python function that returns true ...,['Certainly! Below is a Python function that c...,[' Sure! Here\'s a possible implementation of ...,['Here is a function in Python that checks if ...,['\nHere\'s a Python function that does this:\...,"[""Here's a Python function that checks if a gi...",[' \n[PYTHON]\ndef check_underscore_sequence(i...,"[""Here is a function in Python that checks if ...",0.000464,0.000125,0.004200,0.005424,0.01026,0.000254,0.000154,0.000102,0.000055,0.000126,0.000048,no_model_correct
20695,mbpp.dev.100,['Write a function to find the sum of numbers ...,mbpp,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,['Here\'s a Python function to find the sum of...,['Here is a function to find the sum of number...,['Here is a function that finds the sum of num...,['Here is a Python function to find the sum of...,['Certainly! Below is a Python function that t...,"["" Sure! Here's a function that takes a list o...","['Sure, here is a Python function that does th...","['\n```python\ndef sum_range(lst, start, end):...","[""Here's a Python function th

In [353]:
mmlu_router = routerbench_data.query('eval_name.str.startswith("mb")', engine='python')

In [354]:
mmlu_router = mmlu_router.reset_index(drop = True)

In [355]:
# # Find the common start of all MMLU prompts
# prompts = mmlu_router['prompt'].tolist()
# common_prefix = prompts[0]

# for prompt in prompts[1:]:
#     # Find first differing character
#     for i, (c1, c2) in enumerate(zip(common_prefix, prompt)):
#         if c1 != c2:
#             common_prefix = common_prefix[:i]
#             break
            
# # Remove the common prefix from all prompts
# mmlu_router['prompt'] = mmlu_router['prompt'].str[len(common_prefix):]




In [356]:
# Remove duplicate prompts from mmlu_router
mmlu_router = mmlu_router.drop_duplicates(subset='prompt', keep='first')


In [357]:
mmlu_router.shape

(425, 37)

In [358]:
# Split mmlu_router into train and test sets with fixed sizes
train_data = mmlu_router.sample(n=350, random_state=42)
test_data = mmlu_router.drop(train_data.index).sample(n=75, random_state=42)


In [359]:
# Remove duplicate prompts from train and test data
train_data = train_data.drop_duplicates(subset='prompt', keep='first')
test_data = test_data.drop_duplicates(subset='prompt', keep='first')


In [360]:
from clustering import cluster_sentences

# Extract sentences from train and test data
train_sentences = train_data['prompt'].tolist()
test_sentences = test_data['prompt'].tolist()

# Combine train and test sentences
all_sentences = train_sentences + test_sentences

# Cluster all sentences together
clusters = cluster_sentences(all_sentences, n_clusters=70, show_graph=False, model_name = 'all-distilroberta-v1')

# Create a mapping of sentence to cluster ID
sentence_to_cluster = {}
for cluster_id, cluster_data in clusters.items():
    # Each cluster contains list of [sentence, embedding] pairs, except last element which is avg embedding
    for item in cluster_data[:-1]:  # Exclude the last item (average embedding)
        sentence_to_cluster[item[0]] = cluster_id

# Add cluster IDs to train and test dataframes
train_data['cluster_id'] = train_data['prompt'].map(sentence_to_cluster)
test_data['cluster_id'] = test_data['prompt'].map(sentence_to_cluster)


Batches: 100%|██████████| 14/14 [00:01<00:00, 13.63it/s]


In [361]:
import numpy as np
from clustering import embed_sentence
embeddings = np.array(embed_sentence(all_sentences, model_name = 'all-distilroberta-v1', batch_size = 32))
np.unique(embeddings, axis=0).shape

Batches: 100%|██████████| 14/14 [00:00<00:00, 20.06it/s]


(425, 768)

In [362]:
from sklearn.cluster import KMeans
n_clusters = 100
kmeans = KMeans(n_clusters=n_clusters, random_state=42, max_iter = 4000, tol = 10e-4)
cluster_labels = kmeans.fit_predict(embeddings)

In [363]:
clusters = {i: [] for i in range(n_clusters)}
i=0
for sentence, label in zip(all_sentences, cluster_labels):
    clusters[label].append([sentence,embeddings[i]])
    i+=1

for label in range(n_clusters):
    mask = cluster_labels == label
    cluster_embeddings = embeddings[mask]
    avg_embedding = np.mean(cluster_embeddings, axis=0)
    clusters[label].append(avg_embedding)

In [364]:
# Create a mapping of sentence to cluster ID
sentence_to_cluster = {}
for cluster_id, cluster_data in clusters.items():
    # Each cluster contains list of [sentence, embedding] pairs, except last element which is avg embedding
    for item in cluster_data[:-1]:  # Exclude the last item (average embedding)
        sentence_to_cluster[item[0]] = cluster_id

# Add cluster IDs to train and test dataframes
train_data['cluster_id'] = train_data['prompt'].map(sentence_to_cluster)
test_data['cluster_id'] = test_data['prompt'].map(sentence_to_cluster)

In [365]:
train_data['cluster_id'].value_counts()

cluster_id
29    11
21     9
9      9
13     8
33     7
      ..
61     1
86     1
54     1
75     1
83     1
Name: count, Length: 99, dtype: int64

In [366]:
# Sample 3 prompts from cluster 1 and cluster 2
cluster_1_samples = train_data[train_data['cluster_id'] == 37]['prompt'].sample(n=3, random_state=42)
cluster_2_samples = train_data[train_data['cluster_id'] == 33]['prompt'].sample(n=3, random_state=42)

print("Cluster 1 samples:")
print([prompt[-800:] for prompt in cluster_1_samples])
print("\nCluster 2 samples:")
print([prompt[-800:] for prompt in cluster_2_samples])


Cluster 1 samples:
["['Write a python function to find element at a given index after number of rotations.']", "['Write a python function to find the minimum number of rotations (greater than 0) required to get the same string.']", "['Write a function to rotate a given list by specified number of items to the right direction. https://www.geeksforgeeks.org/python-program-right-rotate-list-n/']"]

Cluster 2 samples:
["['Write a function to get a colon of a tuple.']", "['Write a function to convert a tuple to a string.']", "['Write a function to extract only the rear index element of each string in the given tuple.']"]


In [367]:
# Create a mapping of prompts to their best performing agents
prompt_to_agent = {}

# Process train data
for idx, row in train_data.iterrows():
    prompt = row['prompt']
    # Get the agent with highest score for this prompt
    best_agent = row['oracle_model_to_route_to']
    prompt_to_agent[prompt] = best_agent

# Process test data 
for idx, row in test_data.iterrows():
    prompt = row['prompt']
    # Get the agent with highest score for this prompt
    best_agent = row['oracle_model_to_route_to']
    prompt_to_agent[prompt] = best_agent


In [368]:
# Get unique agents from the data
unique_agents = sorted(list(set(prompt_to_agent.values())))

# Create mapping of agent names to indices
agent_to_idx = {agent: idx for idx, agent in enumerate(unique_agents)}

# Create list of agent indices in order
agents = list(range(len(unique_agents)))

print("Agent mapping:")
for agent, idx in agent_to_idx.items():
    print(f"{agent}: {idx}")


Agent mapping:
WizardLM/WizardLM-13B-V1.2: 0
claude-instant-v1: 1
claude-v1: 2
claude-v2: 3
gpt-3.5-turbo-1106: 4
gpt-4-1106-preview: 5
meta/code-llama-instruct-34b-chat: 6
meta/llama-2-70b-chat: 7
mistralai/mistral-7b-chat: 8
mistralai/mixtral-8x7b-chat: 9
no_model_correct: 10
zero-one-ai/Yi-34B-Chat: 11


In [369]:
from routing import initialize

agents = [i for i in range(12)]
cluster_vectors, agent_vectors = initialize(clusters, agents)

In [370]:
# Create ground truth distributions for each cluster
ground_truth = {}

# First get cluster_id -> prompt mappings
cluster_prompts = {}
for idx, row in train_data.iterrows():
    cluster_id = row['cluster_id']
    prompt = row['prompt']
    if cluster_id not in cluster_prompts:
        cluster_prompts[cluster_id] = []
    cluster_prompts[cluster_id].append(prompt)

# For each cluster, calculate agent distribution
for cluster_id, prompts in cluster_prompts.items():
    # Count occurrences of each agent
    agent_counts = {i: 0 for i in range(12)}
    for prompt in prompts:
        if prompt in prompt_to_agent:
            agent = prompt_to_agent[prompt]
            agent_idx = agent_to_idx[agent]
            agent_counts[agent_idx] += 1
    
    # Convert counts to probabilities
    total = sum(agent_counts.values())
    if total > 0:  # Avoid division by zero
        probs = [agent_counts[i]/total for i in range(12)]
        # Get indices of non-zero probabilities
        nonzero_indices = [i for i, p in enumerate(probs) if p > 0]
        nonzero_probs = [p for p in probs if p > 0]
        ground_truth[cluster_id] = (nonzero_indices, nonzero_probs)

print("Example cluster distributions:")
for cluster_id in list(ground_truth.keys())[:3]:
    print(f"\nCluster {cluster_id}:")
    print(f"Agent indices: {ground_truth[cluster_id][0]}")
    print(f"Probabilities: {ground_truth[cluster_id][1]}")



Example cluster distributions:

Cluster 13:
Agent indices: [0, 8, 9]
Probabilities: [0.5, 0.375, 0.125]

Cluster 90:
Agent indices: [6, 8, 9]
Probabilities: [0.25, 0.5, 0.25]

Cluster 12:
Agent indices: [6, 10, 11]
Probabilities: [0.25, 0.5, 0.25]


In [371]:
print("Example cluster distributions:")
for cluster_id in list(ground_truth.keys()):
    print(f"\nCluster {cluster_id}:")
    # print(f"Agent indices: {ground_truth[cluster_id][0]}")
    print(f"Max Probability: {max(ground_truth[cluster_id][1])}")

Example cluster distributions:

Cluster 13:
Max Probability: 0.5

Cluster 90:
Max Probability: 0.5

Cluster 12:
Max Probability: 0.5

Cluster 22:
Max Probability: 0.75

Cluster 2:
Max Probability: 0.3333333333333333

Cluster 81:
Max Probability: 0.4

Cluster 68:
Max Probability: 0.25

Cluster 5:
Max Probability: 0.5714285714285714

Cluster 11:
Max Probability: 0.5

Cluster 60:
Max Probability: 1.0

Cluster 40:
Max Probability: 0.6666666666666666

Cluster 32:
Max Probability: 0.16666666666666666

Cluster 14:
Max Probability: 0.3333333333333333

Cluster 79:
Max Probability: 0.5

Cluster 9:
Max Probability: 0.4444444444444444

Cluster 72:
Max Probability: 0.5

Cluster 15:
Max Probability: 0.25

Cluster 36:
Max Probability: 0.8

Cluster 99:
Max Probability: 0.5

Cluster 78:
Max Probability: 0.5

Cluster 59:
Max Probability: 0.3333333333333333

Cluster 20:
Max Probability: 0.6666666666666666

Cluster 27:
Max Probability: 0.4

Cluster 98:
Max Probability: 0.5

Cluster 29:
Max Probability: 0.

In [372]:
# Calculate weighted average of max probabilities
total_weight = 0
weighted_sum = 0

for cluster_id in ground_truth.keys():
    # Get number of prompts in this cluster as weight
    cluster_weight = len(cluster_prompts[cluster_id])
    max_prob = max(ground_truth[cluster_id][1])
    
    weighted_sum += max_prob * cluster_weight
    total_weight += cluster_weight

weighted_avg_max_prob = weighted_sum / total_weight
print(f"\nWeighted average maximum probability: {weighted_avg_max_prob:.3f}")



Weighted average maximum probability: 0.506


In [373]:
# Create a version of ground_truth with full probability vectors (including zeros)
ground_truth_format = {}
for cluster_id, (nonzero_indices, nonzero_probs) in ground_truth.items():
    full_probs = [0.0] * 12  # Initialize with zeros for all agents
    for idx, prob in zip(nonzero_indices, nonzero_probs):
        full_probs[idx] = prob
    ground_truth_format[cluster_id] = (range(12), full_probs)


In [374]:
from routing import learn_embeddings

trained_cluster_vectors, trained_agent_vectors = learn_embeddings(
        cluster_vectors, 
        agent_vectors,  
        ground_truth_format,
        epochs = 10,
        learning_rate= 0.1,
        temperature = 10
    )

KeyError: 56

In [375]:
import torch

In [376]:
# Extract embeddings and make predictions on test data
import torch.nn.functional as F

def predict_agent(prompt_embedding, agent_vectors, temperature=0.1):
    # Convert prompt embedding to tensor if it's not already
    if not isinstance(prompt_embedding, torch.Tensor):
        prompt_embedding = torch.tensor(prompt_embedding, dtype=torch.float32)
    
    # Calculate similarities with all agents
    similarities = torch.stack([
        F.cosine_similarity(prompt_embedding.unsqueeze(0),
                          agent_vec.unsqueeze(0))
        for agent_vec in agent_vectors.values()
    ])
    
    # Get predicted agent index
    return similarities.argmax().item()

# Make predictions on test data
correct = 0
total = 0

for _, row in test_data.iterrows():
    cluster_id = row.cluster_id
    true_agent_idx = agent_to_idx[row.oracle_model_to_route_to]

    # Get the prompt embedding (assuming it's the last element)
    prompt_embedding = cluster_vectors[cluster_id]
    
    # Predict agent
    pred_agent_idx = predict_agent(prompt_embedding, trained_agent_vectors)
    
    # Check if prediction is correct
    if pred_agent_idx == true_agent_idx:
        correct += 1
    total += 1

accuracy = correct / total
print(f"\nAccuracy on test data: {accuracy:.2%}")


RuntimeError: The size of tensor a (768) must match the size of tensor b (64) at non-singleton dimension 1

In [377]:
# Count how often rows in same cluster have same eval_name
same_eval_name_count = 0
total_pairs = 0

for _, row in train_data.iterrows():
    # Get all rows with same cluster_id
    cluster_rows = train_data[train_data.cluster_id == row.cluster_id]
    
    # Skip if only 1 row in cluster
    if len(cluster_rows) <= 1:
        continue
        
    # Sample another row from same cluster
    other_row = cluster_rows.sample(n=1).iloc[0]
    
    # Check if eval_names match
    if row.eval_name == other_row.eval_name:
        same_eval_name_count += 1
    total_pairs += 1

cluster_eval_name_similarity = same_eval_name_count / total_pairs
print(f"\nProportion of same-cluster pairs with matching eval_name: {cluster_eval_name_similarity:.2%}")



Proportion of same-cluster pairs with matching eval_name: 100.00%


In [378]:
# Create list of (cluster_id, agent_id) pairs from training data
cluster_agent_pairs = []
for _, row in train_data.iterrows():
    cluster_id = row.cluster_id
    agent_idx = agent_to_idx[row.oracle_model_to_route_to]
    cluster_agent_pairs.append((cluster_id, agent_idx))


In [391]:
import torch
import torch.nn as nn
import torch.nn.functional as F

def train_embeddings(cluster_vectors, agent_vectors, cluster_agent_pairs, 
                     epochs=10, learning_rate=0.05, temperature=0.5):
    """
    Train embeddings using softmax contrastive loss with positive and negative pairs.
    Agent vectors include a learnable bias term, cluster vectors have fixed 0 bias term.
    Adds frequency-based penalties to discourage overuse of common agents.
    """

    optimizer = torch.optim.Adam(
        list(cluster_vectors.values()) + list(agent_vectors.values()),
        lr=learning_rate
    )
    
    # Calculate agent frequencies
    agent_counts = {}
    total_pairs = len(cluster_agent_pairs)
    for _, agent_id in cluster_agent_pairs:
        agent_counts[agent_id] = agent_counts.get(agent_id, 0) + 1
    agent_freqs = {agent_id: count/total_pairs for agent_id, count in agent_counts.items()}
    
    # Determine printing interval safely
    print_interval = max(1, epochs // 10)
    
    for epoch in range(epochs):
        total_loss = 0.0
        
        # Process each cluster
        for cluster_id, cluster_embedding in cluster_vectors.items():
            # Find the positive agent for this cluster
            positive_agent = None
            for c, a in cluster_agent_pairs:
                if c == cluster_id:
                    positive_agent = a
                    break
            if positive_agent is None:
                continue
            
            # Compute cosine similarities for all agents
            similarities = []
            agent_ids = []
            for agent_id, agent_embedding in agent_vectors.items():
                # Compute cosine similarity (returns a tensor of shape (1,))
                sim = F.cosine_similarity(
                    cluster_embedding.unsqueeze(0),
                    agent_embedding.unsqueeze(0)
                )
                # Add frequency penalty
                freq_penalty = agent_freqs.get(agent_id, 0) * 0.1  # Scale factor of 0.1
                sim = sim - freq_penalty
                similarities.append(sim)
                agent_ids.append(agent_id)
            
            # Stack and apply temperature scaling
            logits = torch.stack(similarities) / temperature  # shape: (num_agents,)
            logits = logits.unsqueeze(0)  # shape: (1, num_agents)
            
            # Create target tensor (index of positive agent)
            positive_index = agent_ids.index(positive_agent)
            target = torch.tensor([positive_index], dtype=torch.long, device=logits.device)
            
            # Compute cross entropy loss
            loss = F.cross_entropy(logits, target.unsqueeze(0))
            total_loss += loss.item()
            
            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            # Normalize all vectors with a small epsilon to avoid division by zero
            # Don't normalize the bias terms
            with torch.no_grad():
                for vec in agent_vectors.values():
                    # Only normalize the non-bias terms
                    vec.data[:-1] = vec.data[:-1] / (vec.data.norm(p=2) + 1e-8)
                for vec in cluster_vectors.values():
                    # Only normalize the non-bias terms
                    vec.data[:-1] = vec.data[:-1] / (vec.data[:-1].norm(p=2) + 1e-8)
                    vec.data[-1] = 1.0  # Reset cluster bias term to 0
                    
        if epoch % print_interval == 0:
            print(f"Epoch {epoch}, Loss: {total_loss:.4f}")
    
    return cluster_vectors, agent_vectors

# Example usage:
# Assume `cluster_vectors`, `agent_vectors`, and `cluster_agent_pairs` are defined.
trained_cluster_vectors, trained_agent_vectors = train_embeddings(
    {key: nn.Parameter(torch.cat([torch.normal(0, 1, size=(63,)), torch.ones(1)])) for key in cluster_vectors}, 
    {key: nn.Parameter(torch.normal(0, 1, size=(64,))) for key in agent_vectors},
    cluster_agent_pairs
)


Epoch 0, Loss: 231.5181
Epoch 1, Loss: 186.8882
Epoch 2, Loss: 165.4914
Epoch 3, Loss: 150.1116
Epoch 4, Loss: 140.0291
Epoch 5, Loss: 133.8012
Epoch 6, Loss: 130.1145
Epoch 7, Loss: 127.9464
Epoch 8, Loss: 126.6201
Epoch 9, Loss: 125.7465


In [389]:
# Initialize embeddings
cluster_vecs = {key: nn.Parameter(torch.normal(0, 1, size=(64,))) for key in cluster_vectors}
agent_vecs = {key: nn.Parameter(torch.normal(0, 1, size=(64,))) for key in agent_vectors}

# Calculate weighted average of agent vectors for each cluster
for cluster_id in cluster_vecs:
    # Get the agent distribution for this cluster
    cluster_data = [agent_id for cid, agent_id in cluster_agent_pairs if cid == cluster_id]
    total_samples = len(cluster_data)
    
    # Calculate weights for each agent
    agent_weights = {}
    for agent_id in agent_vecs:
        count = sum(1 for x in cluster_data if x == agent_id)
        agent_weights[agent_id] = 0 if total_samples == 0 else count / total_samples
    
    # Compute weighted average
    weighted_sum = torch.zeros(64)
    weights = torch.tensor([agent_weights[agent_id] for agent_id in agent_vecs.keys()])
    weights = F.softmax(weights / .01, dim=0)  # Apply temperature of 0.1 to logits
    # print(weights)
    for agent_id, weight in zip(agent_vecs.keys(), weights):
        weighted_sum += weight * agent_vecs[agent_id]
        
    # Update cluster vector
    cluster_vecs[cluster_id].data = weighted_sum

trained_cluster_vectors, trained_agent_vectors = cluster_vecs, agent_vecs


In [392]:
# Extract embeddings and make predictions on test data
import torch.nn.functional as F

def predict_agent(prompt_embedding, agent_vectors, temperature=0.1):
    # Convert prompt embedding to tensor if it's not already
    if not isinstance(prompt_embedding, torch.Tensor):
        prompt_embedding = torch.tensor(prompt_embedding, dtype=torch.float32)
    
    # Calculate logits for all agents
    logits = torch.stack([
        torch.matmul(prompt_embedding, agent_vec)
        for agent_vec in agent_vectors.values()
    ])
    
    # Apply temperature scaling and softmax
    probabilities = F.softmax(logits / temperature, dim=0)
    
    # Get predicted agent index
    return probabilities.argmax().item()

# Make predictions on test data
correct = 0
total = 0

for _, row in test_data.iterrows():
    cluster_id = row.cluster_id
    true_agent_idx = agent_to_idx[row.oracle_model_to_route_to]

    prompt_embedding = trained_cluster_vectors[cluster_id]
    
    # Predict agent
    pred_agent_idx = predict_agent(prompt_embedding, trained_agent_vectors)
    # print(pred_agent_idx)
    # Check if prediction is correct
    if pred_agent_idx == true_agent_idx:
        correct += 1
    total += 1

accuracy = correct / total
print(f"\nAccuracy on test data: {accuracy:.2%}")



Accuracy on test data: 28.00%


In [387]:
for i in range(12):
    print(trained_agent_vectors[i][-1])
# Count how often each agent is the best performer
agent_counts = {}
for _, row in test_data.iterrows():
    true_agent_idx = agent_to_idx[row.oracle_model_to_route_to]
    agent_counts[true_agent_idx] = agent_counts.get(true_agent_idx, 0) + 1

# Calculate and print percentages
total_samples = len(test_data)
print("\nAgent selection frequencies in test data:")
for agent_idx, count in sorted(agent_counts.items()):
    percentage = (count / total_samples) * 100
    print(f"Agent {agent_idx}: {percentage:.1f}% ({count} samples)")

tensor(-0.0228, grad_fn=<SelectBackward0>)
tensor(-0.0466, grad_fn=<SelectBackward0>)
tensor(0.3517, grad_fn=<SelectBackward0>)
tensor(0.5380, grad_fn=<SelectBackward0>)
tensor(-0.1804, grad_fn=<SelectBackward0>)
tensor(0.8495, grad_fn=<SelectBackward0>)
tensor(0.0363, grad_fn=<SelectBackward0>)
tensor(-0.6738, grad_fn=<SelectBackward0>)
tensor(-0.1815, grad_fn=<SelectBackward0>)
tensor(-0.8435, grad_fn=<SelectBackward0>)
tensor(-1.1876, grad_fn=<SelectBackward0>)
tensor(1.4942, grad_fn=<SelectBackward0>)

Agent selection frequencies in test data:
Agent 0: 13.3% (10 samples)
Agent 1: 1.3% (1 samples)
Agent 2: 1.3% (1 samples)
Agent 3: 1.3% (1 samples)
Agent 4: 10.7% (8 samples)
Agent 5: 1.3% (1 samples)
Agent 6: 10.7% (8 samples)
Agent 8: 37.3% (28 samples)
Agent 9: 10.7% (8 samples)
Agent 10: 12.0% (9 samples)


In [171]:
# Count clusters where agent 8 is not the best choice
not_best_count = 0
unique_clusters = test_data['cluster_id'].unique()

for cluster_id in unique_clusters:
    cluster_data = test_data[test_data['cluster_id'] == cluster_id]
    agent_performance = {}
    
    # Count performance for each agent in this cluster
    for _, row in cluster_data.iterrows():
        true_agent = agent_to_idx[row.oracle_model_to_route_to]
        agent_performance[true_agent] = agent_performance.get(true_agent, 0) + 1
    
    # Find agent with best performance
    best_agent = max(agent_performance.items(), key=lambda x: x[1])[0]
    
    if best_agent != 8:
        not_best_count += 1

print(f"\nNumber of clusters where agent 8 is not the best choice: {not_best_count}")



Number of clusters where agent 8 is not the best choice: 116


In [251]:
# For each unique cluster, find the most frequent agent and set cluster embedding
for cluster_id in unique_clusters:
    cluster_data = test_data[test_data['cluster_id'] == cluster_id]
    agent_performance = {}
    
    # Count performance for each agent in this cluster
    for _, row in cluster_data.iterrows():
        true_agent = agent_to_idx[row.oracle_model_to_route_to]
        agent_performance[true_agent] = agent_performance.get(true_agent, 0) + 1
    
    # Find agent with best performance
    best_agent = max(agent_performance.items(), key=lambda x: x[1])[0]
    
    # Set cluster embedding to match the best performing agent's embedding
    trained_cluster_vectors[cluster_id] = trained_agent_vectors[best_agent].clone()
